# Example for training Spiking CNN on subset of NMNIST digits

## The problem:
Training digit classifier(0-9) on a subset(1000 training and 100 testing) of NMNIST digit spikes recorded using DVS camera. Just chagne the training list to for full NMNIST training.

## Load proper paths for SLAYER Pytorch source modules

In [1]:
import sys, os
CODE_DIR = os.getcwd()
DATA_DIR = os.path.join(CODE_DIR, '../data')
RESULTS_DIR = os.path.join(CODE_DIR, '../results')
sys.path.append(CODE_DIR + "/slayerPytorch/src")

## Load required modules

SLAYER modules are available as `snn`
* The `spike-layer` module will be available as `snn.layer`.
* The `yaml-parameter` module will be availabe as `snn.params`.
* The `spike-loss` module will be available as `snn.loss`.
* The `spike-classifier` module will be available as `snn.predict`.
* The `spike-IO` module will be available as `snn.io`.


In [2]:
from datetime import datetime
import tqdm.notebook as tqdm
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import slayerSNN as snn
from learningStats import learningStats
from IPython.display import HTML
import zipfile
from torch.utils.tensorboard import SummaryWriter

## Read SNN configuration from yaml file
See the file for all the configuration parameters. This configuration file will be used to describe the SNN. We will ignore the network configuration  describe in the yaml file here.

In [3]:
netParams = snn.params(os.path.join(CODE_DIR, 'network.yaml'))

In [4]:
netParams['training']

{'error': {'type': 'NumSpikes',
  'probSlidingWin': 20,
  'tgtSpikeRegion': {'start': 0, 'stop': 300},
  'tgtSpikeCount': {True: 60, False: 10}},
 'path': {'in': '../data/NMNISTsmall/',
  'train': '../data/NMNISTsmall/train1K.txt',
  'test': '../data/NMNISTsmall/test100.txt'}}

## Extract NMNISTsmall dataset
This is a subset of NMNIST dataset containing first 1000 training samples and first 100 testing samples. The original NMNSIT dataset consists of full MNIST samples converted into spikes using DVS sensor moved in three repeatable saccadic motion. For details and full dataset download links, refer to [https://www.garrickorchard.com/datasets/n-mnist](https://www.garrickorchard.com/datasets/n-mnist)

In [5]:
# extract data from zip only necessary if data is not already extracted
with zipfile.ZipFile(os.path.join(DATA_DIR, 'NMNISTsmall.zip')) as zip_file:
    for member in zip_file.namelist():
        if not os.path.exists(os.path.join(DATA_DIR, member)):
            zip_file.extract(member, DATA_DIR)

## Defne the dataset class
The dataset definition follows standard PyTorch dataset definition.
Internally, it utilizes snn.io modules to read spikes and returns the spike in correct tensor format (CHWT).
* `datasetPath`: the path where the spike files are stored.
* `sampleFile`: the file that contains a list of sample indices and its corresponding clases.
* `samplingTime`: the sampling time (in ms) to bin the spikes.
* `sampleLength`: the length of the sample (in ms)

Note: This is a simple dataset class. A dataset that utilizes the folder hierarchy or xml list is easy to create.

In [6]:
# Dataset definition
class nmnistDataset(Dataset):
    def __init__(self, datasetPath, sampleFile, samplingTime, sampleLength):
        self.path = datasetPath 
        self.samples = np.loadtxt(sampleFile).astype('int')
        self.samplingTime = samplingTime
        self.nTimeBins    = int(sampleLength / samplingTime)

    def __getitem__(self, index):
        inputIndex  = self.samples[index, 0]
        classLabel  = self.samples[index, 1]

        inputSpikes = snn.io.read2Dspikes(
                        self.path + str(inputIndex.item()) + '.bs2'
                        ).toSpikeTensor(torch.zeros((2,34,34,self.nTimeBins)),
                        samplingTime=self.samplingTime)
#         sparseInputSpikes = torch.sparse_coo_tensor([inputSpikes.p, inputSpikes.y, inputSpikes.x, inputSpikes.t],
#                                                         np.ones_like(inputSpikes.p), (2, 34, 34, 306))

        desiredClass = torch.zeros((10, 1, 1, 1))
        desiredClass[classLabel,...] = 1
        return inputSpikes, desiredClass, classLabel

    def __len__(self):
        return self.samples.shape[0]

In [7]:
# Dataset definition
class nmnistDatasetNoisy(nmnistDataset):
    def __init__(self, datasetPath, sampleFile, samplingTime, sampleLength, percentNoise=0.001):
        super().__init__(datasetPath, sampleFile, samplingTime, sampleLength)
        self.numNoise = int(percentNoise*2*34*34*300)
        # generate binary (boolean) noise with percentNoise 1s
#         self.noise = torch.cuda.FloatTensor(len(self.samples), 2, 34, 34, self.nTimeBins).uniform_() < percentNoise
    def __getitem__(self, index):
        inputSpikes, desiredClass, classLabel = super().__getitem__(index)
#         for i in range(int(self.percentNoise*torch.numel(inputSpikes))):
        p = torch.randint(0, 2, (self.numNoise,))
        x, y = torch.randint(0, 34, (2,self.numNoise))
        t = torch.randint(0, 300, (self.numNoise,))
        inputSpikes[p, x, y, t] = 1-inputSpikes[p, x, y, t]
        # noise = torch.FloatTensor(2, 34, 34, self.nTimeBins).uniform_() < self.percentNoise #self.noise[index] # get pre-generated noise for this example
        # inputSpikes[noise] = 1-inputSpikes[noise] # invert 1s and 0s where noise=True
        return inputSpikes, desiredClass, classLabel

## Visualize the spike data

In [8]:
trainingSet = nmnistDataset(datasetPath =netParams['training']['path']['in'], 
                            sampleFile  =netParams['training']['path']['train'],
                            samplingTime=netParams['simulation']['Ts'],
                            sampleLength=netParams['simulation']['tSample'])

In [9]:
len(trainingSet.samples)

1000

In [10]:
input, target, label = trainingSet[0]
anim = snn.io.animTD(snn.io.spikeArrayToEvent(input.reshape((2, 34, 34, -1)).cpu().data.numpy()))
# anim = snn.io.animTD(snn.io.spikeArrayToEvent(input.to_dense().reshape((2, 34, 34, -1)).cpu().data.numpy()))
HTML(anim.to_jshtml())

In [11]:
torch.sum(input)

tensor(4670.)

In [12]:
34*34*300*2

693600

In [13]:
4670/693600

0.006732987312572087

## Noisy data

In [14]:
trainingSetNoisy = nmnistDatasetNoisy(datasetPath =netParams['training']['path']['in'], 
                            sampleFile  =netParams['training']['path']['train'],
                            samplingTime=netParams['simulation']['Ts'],
                            sampleLength=netParams['simulation']['tSample'],
                            percentNoise = 0.001)

In [15]:
input2, target2, label2 = trainingSetNoisy[0]
anim = snn.io.animTD(snn.io.spikeArrayToEvent(input2.reshape((2, 34, 34, -1)).cpu().data.numpy()))
HTML(anim.to_jshtml())

In [16]:
# Delete the rogue temp-file
try:
    os.remove('None0000000.png')
except FileNotFoundError:
    pass

## Define the network
The network definition follows similar style as standard PyTorch network definition, but it utilizes snn modules.

In [17]:
class PSPLayer(torch.nn.Module):
    def __init__(self, netParams=netParams):
        super(PSPLayer, self).__init__()
        self.slayer = snn.layer(netParams['neuron'], netParams['simulation'])
    def forward(self, x):
        return self.slayer.psp(x)

In [18]:
class ApplySpikeLayer(torch.nn.Module):
    def __init__(self, netParams=netParams):
        super(ApplySpikeLayer, self).__init__()
        self.slayer = snn.layer(netParams['neuron'], netParams['simulation'])
    def forward(self, x):
        return self.slayer.spike(x)

In [32]:
class SpikingNetwork(torch.nn.Module):
    def __init__(self, netParams=netParams):
        super(SpikingNetwork, self).__init__()
        # initialize slayer
        self.slayer = snn.layer(netParams['neuron'], netParams['simulation'])
        self.psplayer = PSPLayer(netParams)
        self.applyspikelayer = ApplySpikeLayer(netParams)
        # self.slayer = slayer
        
    def forward(self, spikeInput):
#         x = spikeInput
#         for layer in self.layers:
#             x = self.slayer.spike(layer(self.slayer.psp(x)))
#         return x
        return self.layers(spikeInput)
    
    def setLayers(self, layers):
        if isinstance(layers, list):
            self.layers = []
            for layer in layers:
                self.layers.append(self.psplayer)
                self.layers.append(layer)
                self.layers.append(self.applyspikelayer)
            self.layers = torch.nn.Sequential(*self.layers)
        else:
            raise Exception("layers should be a list of layers")
    def readyTraining(self, device, deviceIds):
        # Create network instance.
        # net = Network(netParams).to(device)
        # Split the network to run over multiple GPUs
        net = torch.nn.DataParallel(self.to(device), device_ids=deviceIds)
        
        # Create snn loss instance.
        error = snn.loss(netParams).to(device)

        # Define optimizer module.
        optimizer = torch.optim.Adam(net.parameters(), lr = 0.01, amsgrad = True)

        # Dataset and dataLoader instances.
        trainingSet = nmnistDatasetNoisy(datasetPath =netParams['training']['path']['in'], 
                                    sampleFile  =netParams['training']['path']['train'],
                                    samplingTime=netParams['simulation']['Ts'],
                                    sampleLength=netParams['simulation']['tSample'],
                                    percentNoise=0)
        trainLoader = DataLoader(dataset=trainingSet, batch_size=8, shuffle=True, num_workers=4)

        testingSet = nmnistDatasetNoisy(datasetPath  =netParams['training']['path']['in'], 
                                    sampleFile  =netParams['training']['path']['test'],
                                    samplingTime=netParams['simulation']['Ts'],
                                    sampleLength=netParams['simulation']['tSample'],
                                    percentNoise=0)
        testLoader = DataLoader(dataset=testingSet, batch_size=8, shuffle=True, num_workers=4)

        # Learning stats instance.
        stats = learningStats()
        
        self.netVars = {
            'net': net,
            'error': error,
            'optimizer': optimizer,
            'trainingSet': trainingSet,
            'trainLoader': trainLoader,
            'testingSet': testingSet,
            'testLoader': testLoader,
            'stats': stats
        }
    def doTrain(self, device, epochcount=1, writer=SummaryWriter()):
        stats = self.netVars['stats']
        net = self.netVars['net']
        trainLoader = self.netVars['trainLoader']
        testLoader = self.netVars['testLoader']
        error = self.netVars['error']
        optimizer = self.netVars['optimizer']
        for epoch in tqdm.trange(epochcount, desc='epochs', leave=False):
            # Reset training stats.
            stats.training.reset()
            tSt = datetime.now()

            # Training loop.
            for i, (input, target, label) in enumerate(tqdm.tqdm(trainLoader, desc='training loop', leave=False), 0):
                # Move the input and target to correct GPU.
                input  = input.to(device)
                target = target.to(device) 

                # Forward pass of the network.
                output = net.forward(input)

                # Gather the training stats.
                stats.training.correctSamples += torch.sum( snn.predict.getClass(output) == label ).data.item()
                stats.training.numSamples     += len(label)

                # Calculate loss.
                loss = error.numSpikes(output, target)

                # Reset gradients to zero.
                optimizer.zero_grad()

                # Backward pass of the network.
                loss.backward()

                # Update weights.
                optimizer.step()

                # Gather training loss stats.
                stats.training.lossSum += loss.cpu().data.item()

                # Display training stats. (Suitable for normal python implementation)
                # stats.print(epoch, i, (datetime.now() - tSt).total_seconds())

            # Update training stats.
            stats.training.update()
            # Reset testing stats.
            stats.testing.reset()

            # Testing loop.
            # Same steps as Training loops except loss backpropagation and weight update.
            for i, (input, target, label) in tqdm(enumerate(testLoader, 0), desc='testing loop', leave=False):
                input  = input.to(device)
                target = target.to(device) 

                output = net.forward(input)

                stats.testing.correctSamples += torch.sum( snn.predict.getClass(output) == label ).data.item()
                stats.testing.numSamples     += len(label)

                loss = error.numSpikes(output, target)
                stats.testing.lossSum += loss.cpu().data.item()
                # stats.print(epoch, i)

            # Update testing stats.
            stats.testing.update()
            writer.add_scalar('Accuracy/train', stats.training.accuracy(), epoch)
            writer.add_scalar('Accuracy/test',  stats.testing.accuracy(),  epoch)
            writer.add_scalar('Loss/train',     stats.training.loss(),     epoch)
            writer.add_scalar('Loss/test',      stats.testing.loss(),      epoch)

            # if epoch%10==0:  stats.print(epoch, timeElapsed=(datetime.now() - tSt).total_seconds())
        writer.close()

# create several networks with different architectures

In [33]:
Networks = []

In [34]:
Networks.append(SpikingNetwork())
slayer = Networks[0].slayer
Networks[0].setLayers([
            slayer.conv(2, 16, 5, padding=1),
            slayer.pool(2),
            slayer.conv(16, 32, 3, padding=1),
            slayer.pool(2),
            slayer.conv(32, 64, 3, padding=1),
            slayer.dense((8,8,64), 10)
        ])

In [35]:
Networks.append(SpikingNetwork())
slayer = Networks[1].slayer
Networks[1].setLayers([
            slayer.conv(2, 6, 5, padding=1),
            slayer.pool(2),
            slayer.conv(6, 12, 3, padding=1),
            slayer.pool(2),
            slayer.conv(12, 12, 3, padding=1),
            slayer.dense((8,8,12), 10)
        ])

In [36]:
Networks.append(SpikingNetwork())
slayer = Networks[2].slayer
Networks[2].setLayers([
            slayer.conv(2, 6, 5, padding=1),
            slayer.pool(4),
            slayer.conv(6, 12, 3, padding=1),
            slayer.pool(4),
            slayer.conv(12, 12, 3, padding=1),
            slayer.dense((2,2,12), 10)
        ])

# Train the network
Train the network for 100 epochs.

In [37]:
# first move to a new directory to not overwrite anything
dt_string = datetime.now().strftime("%d-%m-%Y_%Hh%Mm%S")
dirname = dt_string
i = 0
while os.path.exists(os.path.join(RESULTS_DIR, dirname)):
    dirname = dt_string + '_' + str(++i)
os.mkdir(os.path.join(RESULTS_DIR, dirname))

In [38]:
# Define the cuda device to run the code on.
# device = torch.device('cuda')
# Use multiple GPU's if available
device = torch.device('cuda:0')#:2') # should be the first GPU of deviceIDs 
deviceIds = [0]#2, 3]

In [39]:
for i, Network in enumerate(tqdm.tqdm(Networks, desc='network')):
    Network.readyTraining(device, deviceIds)
    Network.doTrain(device, epochcount=100, writer=SummaryWriter(os.path.join(RESULTS_DIR,'data',dirname,"modelset5_noise/",str(i)), comment='model'+str(i)))

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## ~~Plot the Results~~